# nnUNet
Prepare input data to launch nnUNet into the CHUV cluster.

Check this repo: https://github.com/PeterMcGor/nnUNet/tree/docker_and_singularity

## Read subjects from txt (train, validation, test)

In [6]:
import os, sys, shutil, random, tempfile, glob, logging, math, csv

txt_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/'
data_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/' # train and val

# Train subjects
train_file = txt_dir + 'train_subjects.txt' # train subjects list file
train_subs = [] # empty list to read list from a file
# open file and read the content in a list
with open(train_file, 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        train_subs.append(x)
# display list
print(train_subs)

# Val subjects
val_file = txt_dir + 'val_subjects.txt' # val subjects list file
val_subs = [] # empty list to read list from a file
# open file and read the content in a list
with open(val_file, 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        val_subs.append(x)
# display list
print(val_subs)

# Test
file = txt_dir + 'test_subjects.txt' # test subjects list file
test_subs = [] # empty list to read list from a file
# open file and read the content in a list
with open(file, 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        test_subs.append(x)
# display list
print(test_subs)

['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-10', 'sub-11', 'sub-12', 'sub-16', 'sub-17', 'sub-18', 'sub-20', 'sub-22', 'sub-23', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 'sub-35']
['sub-09', 'sub-15', 'sub-19', 'sub-24']
['sub-13', 'sub-14', 'sub-21', 'sub-34']


## Copy subjects into output dir and change their names

In [8]:
# Output paths
output_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet_raw_data/Task313_Eye/'
imagesTr_path = f'{output_dir}imagesTr/'
labelsTr_path = f'{output_dir}labelsTr/'
imagesTs_path = f'{output_dir}imagesTs/'

# Adding val subjects into train set for the nnUNet
train_val_subs = train_subs + val_subs

# Lengths of sets
len_test_subs = len(test_subs)
print(f'Number of test subjects: {len_test_subs}')
len_train_val_subs = len(train_val_subs)
print(f'Number of train subjects: {len_train_val_subs}')

# Toggle one subject (to test) or the entire set
TOGGLE_ENTIRE_SET = True


# Test set
for j in range(len_test_subs):
    
    # Input path
    input_t1 = data_dir + 'a123/' + test_subs[j] + '/input/' + test_subs[j] + '_T1_aff.nii.gz'
    print(f'Path T1: {input_t1}')

    # New name
    new_name_im = f'AEye_{j+1:03d}_0000.nii.gz'
    print(f'New name image T1: {new_name_im}')

    # Copy
    shutil.copy(input_t1, f'{imagesTs_path}{new_name_im}')

    if j==0 and TOGGLE_ENTIRE_SET==False:
        break

# Train set
for i in range(len_train_val_subs):
    
    # Input paths
    input_t1 = data_dir + 'a123/' + train_val_subs[i] + '/input/' + train_val_subs[i] + '_T1_aff.nii.gz'
    print(f'Path T1: {input_t1}')
    input_labels = data_dir + 'a123/' + train_val_subs[i] + '/input/' + train_val_subs[i] + '_labels_aff.nii.gz'
    print(f'Path labels: {input_labels}')

    # New name
    new_name_im = f'AEye_{i+1+len_test_subs:03d}_0000.nii.gz'
    print(f'New name image T1: {imagesTr_path}{new_name_im}')
    new_name_lab = f'AEye_{i+1+len_test_subs:03d}.nii.gz'
    print(f'New name label: {labelsTr_path}{new_name_lab}')

    # Copy
    shutil.copy(input_t1, f'{imagesTr_path}{new_name_im}')
    shutil.copy(input_labels, f'{labelsTr_path}{new_name_lab}')

    if i==0 and TOGGLE_ENTIRE_SET==False:
        break

# Dealing with files in that folder
for f in glob.glob(f'{imagesTr_path}/*0001.nii.gz'):
    os.remove(f)

Number of test subjects: 4
Number of train subjects: 31
Path T1: /mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/a123/sub-13/input/sub-13_T1_aff.nii.gz
New name image T1: AEye_001_0000.nii.gz
Path T1: /mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/a123/sub-14/input/sub-14_T1_aff.nii.gz
New name image T1: AEye_002_0000.nii.gz
Path T1: /mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/a123/sub-21/input/sub-21_T1_aff.nii.gz
New name image T1: AEye_003_0000.nii.gz
Path T1: /mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/a123/sub-34/input/sub-34_T1_aff.nii.gz
New name image T1: AEye_004_0000.nii.gz
Path T1: /mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/a123/sub-01/input/sub-01_T1_aff.nii.gz
Path labels: /mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/a123/sub-01/input/sub-01_labels_aff.nii.gz
New name image T1: /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet_raw_data/T

## Generate dataset json

In [9]:
from nnunet.dataset_conversion.utils import generate_dataset_json

# Output paths
output_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet_raw_data/Task313_Eye/'
imagesTr_path = f'{output_dir}imagesTr/'
labelsTr_path = f'{output_dir}labelsTr/'
imagesTs_path = f'{output_dir}imagesTs/'

generate_dataset_json(output_file=os.path.join(output_dir, 'dataset.json'), imagesTr_dir=imagesTr_path, imagesTs_dir=imagesTs_path,
                        modalities=('T1w',), labels={0: 'background', 1: 'lens', 2: 'globe', 3: 'optic nerve',
                        4: 'intraconal fat', 5: 'extraconal fat', 6: 'lateral rectus muscle', 
                        7: 'medial rectus muscle', 8: 'inferior rectus muscle', 9: 'superior rectus muscle'},
                        dataset_name='AEyeDataset')

## Simple sample

In [ ]:
# docker run --gpus device=0 --shm-size=10gb -v /home/pmacias/nnunet_root_test:/opt/nnunet_resources -it petermcgor/nnunet:0.0.1 bash
# nnUNet_plan_and_preprocess -t XXX --verify_dataset_integrity
# singularity run --bind /home/{youruser}/nnUNet_test_data:/opt/nnunet_resources  --nv docker://petermcgor/nnunet:0.0.1 nnUNet_train 2d nnUNetTrainerV2 Task002_Heart 1 --npz

# Use decathlon datasets
# sudo docker run --gpus device=0 --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_convert_decathlon_task -i /home/jaimebarranco/Desktop/nnUNet/nnUNet_raw_data/Task02_Heart -p NUM_PROCESSES (8 for example)

# Verify dataset integrity
# sudo docker run --gpus device=0 --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_plan_and_preprocess -t 2 --verify_dataset_integrity
# sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_plan_and_preprocess -t 313 --verify_dataset_integrity

# To enter the Docker machine (-it for interactive and bash for the machine)
# sudo docker run --gpus device=0 -it --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnUNet_resources petermcgor/nnunet:0.0.1 bash

# Training
# sudo docker run --gpus device=0 --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_MYTASK FOLD --npz
# sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_train 3d_fullres nnUNetTrainerV2 Task313_Eye 0
